## Lib Import

In [ ]:
# Import Pipeline
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# Import filter/screen
from quantopian.pipeline.filters import QTradableStocksUS

# Import data
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.factset import Fundamentals as ff
from quantopian.pipeline.data.morningstar import Fundamentals as mf

# Import classifier
from quantopian.pipeline.classifiers.fundamentals import Sector

# Import Alphalens
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

In [ ]:
# Constants
start_date = '2018-1-1'
end_date = '2020-6-14'

factor_name = 'fcf_yld'

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

group_neutral = True

long_short = True

In [ ]:
# Create pipeline function
from quantopian.pipeline.filters import  StaticAssets
from quantopian.pipeline.factors import DailyReturns


def make_pipeline():
    # Get our primary data
    u = StaticAssets(symbols(['AAPL', 'MSFT','AMZN','FB']))
    #u = StaticAssets(symbols(['AAPL' ]))
    
    sector = Sector()
    
    fcf = mf.free_cash_flow.latest
    ev = mf.enterprise_va aalue.latest
    
    # Calculate factor
    
    fcf_yld = fcf / ev
    
    #Create screens
    
    factor_screen = fcf_yld.notnull()
    
    sector_screen = sector.notnull()
    
    # Return pipeline
    
    return Pipeline(
        columns={
            factor_name: fcf_yld,
            #factor_name: fcf,
            'close': DailyReturns(),
            'sector': sector
        },
        screen  = u
        #screen=QTradableStocksUS() & factor_screen & sector_screen,
    )

In [ ]:
# Get factor data from pipeline
factor_data = run_pipeline(make_pipeline(), start_date, end_date)

In [ ]:
factor_data.tail(100)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dates = factor_data.index.levels[0]
idx = pd.IndexSlice
for ticker in ['AAPL', 'MSFT','AMZN','FB']:
    plt.plot(dates, factor_data.loc[idx[:, symbols(ticker)], :]['fcf_yld'],label=ticker ) 
plt.legend()
plt.title('Historical T CashFlow (in billions)')
plt.show()

In [ ]:
dates = factor_data.index.levels[0]
idx = pd.IndexSlice
for ticker in ['AAPL', 'MSFT','AMZN','FB']:
    plt.plot(dates, factor_data.loc[idx[:, symbols(ticker)], :]['close'].cumsum() ,label=ticker ) 
plt.legend( loc='upper left')
plt.title('Historical T CashFlow (in billions)')
plt.show( )

In [ ]:
factor_data.plot()

In [ ]:
# Get pricing data
pricing_data = get_pricing(factor_data.index.levels[1], start_date, end_date, fields='open_price')

In [ ]:
pricing_data.head()

In [ ]:
# Get factor and returns data
merged_data = get_clean_factor_and_forward_returns(
    factor=factor_data[factor_name],
    prices=pricing_data,
    periods=(1,5,10),
    quantiles=5,
    groupby=factor_data['sector'],
    groupby_labels = sector_labels,
    binning_by_group = group_neutral,
)

In [ ]:
merged_data.tail()

In [ ]:
# Create tear-sheet to analyse
create_full_tear_sheet(merged_data, long_short=long_short, by_group=True, group_neutral=group_neutral)